# VERIFICA LEGGE DI MALUS:

$I(\alpha) = I_0\cos^2(\alpha)$

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import sys

dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')

from interpolazione3 import Interpolazione, final_val

dir_path = os.path.abspath('')
FILE = dir_path + '/data/' + 'malus.csv'
fr = pd.read_csv(FILE)

In [2]:
# import dati da csv

α = np.array(fr['alpha (deg)']) * np.pi / 180 # rad
V = np.array(fr['V (Volt)']) # Volt
# I = np.array(fr['I (mA)'])[0:11] # Ampere
# sigmaI = np.array(fr['sigmaI'])[0:11] # Ampere
sigmaV = np.array(fr['sigmaV']) # Volt
sigmaα = np.ones_like(α)*np.pi/180 # rad

Abbiamo due modelli possibili:
- Il nostro sensore misura l'intensità del fascio $I$
- Il nostro sensore misura l'ampiezza del campo elettrico $E$

Sappiamo che il fascio si tratta di microonde polarizzate linearmente

cerchiamo inoltre l'angolo in cui la polarizzazione è massima variando l'angolo del ricevitore

In [3]:
def legge_malus(α,I0,θ_0):
    return I0 * np.cos(α + θ_0)**2

def campo_polarizzato(α,E0,θ_0):
    return E0 * np.abs(np.cos(α + θ_0))

malus_E = Interpolazione(α, V,sigmaV,sigmaα,campo_polarizzato,guess=[1,0],names=['E0','θ'])
print(malus_E)
# malus_I = Interpolazione(α, I, campo_polarizzato, sigmaI,[0,0],names=['I0','offset'])

TypeError: Interpolazione.__init__() got an unexpected keyword argument 'guess'

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

# Plot for V

ax.errorbar(α, V, yerr=sigmaV,xerr=np.pi/180*np.sqrt(2)*2, fmt='.', label='data')

x = np.linspace(-np.pi,np.pi, 10000)
y = campo_polarizzato(x, *malus_E.bval)
ax.plot(x,y, label='fit')


ax.set_xticks([-np.pi,-3*np.pi/4,-np.pi/2,-np.pi/4,0,np.pi/4, np.pi/2, 3*np.pi/4, np.pi])
ax.set_xticklabels(['$-\pi$','$-3\pi/4$','$-\pi/2$','$-\pi/4$','$0$','$\pi/4$','$\pi/2$','$3\pi/4$','$\pi$'])
ax.set_xlabel('α')
ax.set_ylabel('I / V')
ax.set_title('I and V vs α')
ax.legend()

plt.tight_layout()
plt.show()



le andature di tensione e di corrente seguono la legge $|\cos(x)|$, i nostri rilevatori leggono quindi l'ampiezza del campo elettrico